In [67]:
import numpy as np
import pandas as pd
import math
import scipy
import sklearn
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, DotProduct
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn.gaussian_process.kernels as kernels
from scipy.stats import spearmanr, pearsonr
import scipy.stats

In [25]:
Data = pd.read_csv('Data.csv' , )
X = Data['x']
X  = X.values.reshape((10, 1))
Y = Data['y']

In [3]:
def getPrediction(X, Y, X1):
    gp = GaussianProcessRegressor(kernel=kernels.DotProduct(), optimizer='fmin_l_bfgs_b' , alpha=1e-10, normalize_y=True)
    gp.fit(X, Y)
    return gp.predict(X1)

In [4]:
def getFullCorrelation(X, T, Y, num_t):
    PredictionFunctions = np.zeros((len(X) , num_t ))
    Correlations = np.zeros((num_t, num_t))
    for t in range(0, num_t):
        X_t = X[T == t]
        Y_t = Y[T == t]
        PredictionFunctions[:, t] = getPrediction(X_t, Y_t, X)
    for i in range(0, num_t):
        for j in range(i, num_t):
            Correlations[i, j] = np.cov(PredictionFunctions[:, i], PredictionFunctions[:, j])[0, 1]
            Correlations[j , i] = Correlations[i, j]
    return PredictionFunctions, Correlations

In [142]:
T = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
PredictionFunctions, Correlations = getFullCorrelation(X, T, Y, 2)
print(PredictionFunctions)
print(Correlations)

[[ 2.06600012  3.98099934]
 [ 3.05350012  5.98599915]
 [ 4.04099989  7.99099849]
 [ 5.02850013  9.99599783]
 [ 6.01600037 12.00099908]
 [ 7.00349966 14.00599842]
 [ 7.99100086 16.01099776]
 [ 8.97849919 18.0159971 ]
 [ 9.96600039 20.02099835]
 [10.95349968 22.02599959]]
[[ 8.93893185 18.14942602]
 [18.14942602 36.85022667]]


# MTGP with Normalizing (not working now - some error)

In [48]:
# def kernelfunction(a, b):
#     return np.dot(a, b) + 0.1
    
# def MTGPPredict(x_new, t_new, X, T, alpha, Cn_inv, Correlations, sigma2):
#     N = len(X)
#     Knew = np.zeros(N)
#     for i in range(N):
#         Knew[i] = kernelfunction(x_new, X[i])*Correlations[t_new , T[i]]
#     mean = np.dot(Knew, alpha)
#     var1 = np.dot(np.dot(Knew, Cn_inv) , Knew)
#     var =  kernelfunction(x_new, x_new)*Correlations[t_new , t_new] + sigma2 - var1
#     return mean, var

# def MTGP(X, T, Y_raw, X_pred, T_pred, Correlations , sigma2 , num_task):
#     N = len(X)
    
#     Y_task_mean = np.zeros(num_task)
#     Y_task_var = np.zeros(num_task)
#     Y = np.zeros(N)
#     for t in range(0, num_task):
#         Y_task_mean[t] = np.mean(Y_raw[T == t])
#         Y_task_var[t] = np.std(Y_raw[T == t])
#         if (Y_task_var[t] !=0):
#             Y[T == t] = (Y_raw[T == t] - Y_task_mean[t])/Y_task_var[t]
    
#     Cn = np.zeros((N, N))
    
#     for i in range(0, N):
#         for j in range(i, N):
#             Cn[i, j] = kernelfunction(X[i],  X[j])*Correlations[T[i] , T[j]]
#             Cn[j , i] = Cn[i , j]
#     Cn = Cn + sigma2*np.identity(N)
#     Cn_inv = np.linalg.inv(Cn)
#     alpha = np.dot(Cn_inv, Y)
#     Y_pred_mean = np.zeros(N)
#     Y_pred_var = np.zeros(N)
    
#     for i in range(N):
#         Y_pred_mean[i], Y_pred_var[i] = MTGPPredict(X_pred[i] , T_pred[i] , X, T,alpha, Cn_inv, Correlations , sigma2 )
        
#     for t in range(0, num_task):
#         if (Y_task_var[t] !=0):
#             Y_pred_mean[T == t] = (Y_pred_mean[T == t]*Y_task_var[t] + Y_task_mean[t])
    
#     print(np.array(Y_pred_mean))

#     return Y_pred_mean, Y_pred_var

# MTGP without normalizing y

In [214]:
# def kernelfunction(a, b):
#     return np.dot(a, b) + 0.1

def kernelfunction(a, b):
    return  a*a*b*b + a*b + 1
    
def MTGPPredict(x_new, t_new, X, T, alpha, Cn_inv, Correlations, sigma2):
    N = len(X)
    Knew = np.zeros(N)
    for i in range(N):
        Knew[i] = kernelfunction(x_new, X[i])*Correlations[t_new , T[i]]
    mean = np.dot(Knew, alpha)
    var1 = np.dot(np.dot(Knew, Cn_inv) , Knew)
    var =  kernelfunction(x_new, x_new)*Correlations[t_new , t_new] + sigma2 - var1
    return mean, var

def MTGP(X, T, Y, X_pred, T_pred, Correlations , sigma2 , num_task):
    N = len(X)    
    Cn = np.zeros((N, N))
    
    for i in range(0, N):
        for j in range(i, N):
            Cn[i, j] = kernelfunction(X[i],  X[j])*Correlations[T[i] , T[j]]
            Cn[j , i] = Cn[i , j]
    Cn = Cn + sigma2*np.identity(N)
    Cn_inv = np.linalg.inv(Cn)
    alpha = np.dot(Cn_inv, Y)
    Y_pred_mean = np.zeros(len(X_pred))
    Y_pred_var = np.zeros(len(X_pred))
    
    for i in range(len(X_pred)):
        Y_pred_mean[i], Y_pred_var[i] = MTGPPredict(X_pred[i] , T_pred[i] , X, T,alpha, Cn_inv, Correlations , sigma2 )
    return Y_pred_mean, Y_pred_var

In [205]:
def getProb(Y_true, Y_pred_mean, Y_pred_var):
    prob = np.zeros(len(Y_true))
    for i in range(len(Y_true)):
        p = scipy.stats.norm(Y_pred_mean[i], Y_pred_var[i]).pdf(Y_true[i])
        prob[i] = p 
    return prob
    

def getMTGPTask(X, curr_T, Y , sigma2, num_task):
    PredictionFunctions, Correlations = getFullCorrelation(X, T, Y, num_task)
    X_pred = np.repeat(X, num_task)
    T_pred = np.repeat(np.repeat([np.arange(num_task)], len(X), axis=0), 1)
    Y_pred_mean, Y_pred_var = MTGP(X, curr_T, Y, X_pred, T_pred, Correlations , sigma2 , num_task)
    Y_true = np.repeat(np.array(Y), num_task)
    Prob_Task = getProb(Y_true, Y_pred_mean, Y_pred_var)
    new_Tasks = np.zeros(len(X))
    for i in range(len(X)):
        new_Tasks[i] = np.argmax(Prob_Task[ i*num_task : (i+1)*num_task ])
    return new_Tasks

    
    

In [255]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth = 10, random_state = 2404)

T = np.array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

for i in range(5):
    T = getMTGPTask(X, T, Y, 0.1, len(set(T))).astype(int)
    clf.fit(X, T)
    T  = clf.predict(X)
    print(T)


Y_pred_mean, Y_pred_var = MTGP(X, T, Y, X, T , Correlations , 0.1, len(set(T)))

prediction = pd.DataFrame(Y_pred_mean, columns=['pred'])
prediction['True'] = Y
print(prediction)
diff = prediction['pred'] - prediction['True']
loss = math.sqrt(np.dot(diff, diff)/len(prediction['pred']))
print(T)
print('loss = ' ,loss)

In [84]:
scipy.stats.norm(0, 1).pdf(0)

0.3989422804014327

# Test for MTGP

In [49]:
c = np.zeros(10)
c = c.astype(int)
MTGP(X, T, Y, X, c , Correlations , 0.000001, 2)
c = c + 1
MTGP(X, T, Y, X, c , Correlations , 0.000001, 2)


[ 1.9910133   2.97648597  3.96195863  4.94743129  5.93290395  6.91837659
  7.90384927  8.88932194  9.87479458 10.86026723]
[ 4.04251163  6.04339392  8.04427622 10.04515854 12.04604085 14.04692316
 16.04780548 18.04868782 20.04957012 22.05045242]


In [266]:
def genSine(low, high, count):
    noise_mean = 0
    noise_variance = 0.1
    x = np.linspace(low, high, num=count)
    y = np.sin(x)+np.random.normal(noise_mean, noise_variance, count)
    return x.reshape((count,1)), y

data_x, data_y = genSine(0, 2*np.pi, 100)
print(data_x.shape, data_y.shape)

Data = pd.DataFrame(data_x )
Data.columns = ['x']
Data['y'] = data_y
Data_train = Data.sample(frac=0.5, random_state=2404)

X = Data_train['x']
X  = X.values.reshape((50, 1))
Y = Data_train['y']

random.seed(2404)
T = np.random.randint(3, size=len(X))

for i in range(10):
    T = getMTGPTask(X, T, Y, 0.1, len(set(T))).astype(int)
    clf.fit(X, T)
    T  = clf.predict(X)
    print(T)

PredictionFunctions, Correlations = getFullCorrelation(X, T, Y, 3)

Y_pred_mean, Y_pred_var = MTGP(X, T, Y, X, T , Correlations , 0.1, len(set(T)))

prediction = pd.DataFrame(Y_pred_mean, columns=['pred'])
prediction['True'] = Data_train['y']
print(prediction)
diff = prediction['pred'] - prediction['True']
loss = math.sqrt(np.dot(diff, diff)/len(prediction['pred']))
print(T)
print('loss = ' ,loss)




(100, 1) (100,)
[0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 2 0 2 0 2 2 0 0 0 2 2 2 2 0 1 0 2 0 0
 2 2 0 0 0 0 0 1 0 2 0 2 0]
[0 0 0 2 0 0 2 0 0 0 1 1 0 0 1 0 0 2 0 1 2 0 2 1 0 0 0 2 2 2 2 0 0 0 2 0 0
 1 0 0 0 0 1 0 0 0 0 0 2 0]
[0 0 0 1 0 0 2 0 0 0 2 0 0 0 0 1 0 2 1 2 2 0 2 1 0 0 0 2 2 2 2 0 0 0 2 0 0
 1 1 0 0 0 0 0 0 1 1 0 2 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 1 0 0 0 2 2 2 2 0 1 0 2 0 0
 1 1 0 0 0 0 0 1 1 1 0 2 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 2 0 0 0 2 1 2 2 0 1 0 2 0 0
 1 1 0 0 0 0 0 1 1 1 0 2 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 2 0 0 0 2 1 2 2 0 1 0 2 0 0
 1 1 0 0 0 0 0 1 1 1 0 2 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 2 0 0 0 2 1 2 2 0 1 0 2 0 0
 1 1 0 0 0 0 0 1 1 1 0 2 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 2 0 0 0 2 1 2 2 0 1 0 2 0 0
 1 1 0 0 0 0 0 1 1 1 0 2 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 2 0 0 0 2 1 2 2 0 1 0 2 0 0
 1 1 0 0 0 0 0 1 1 1 0 2 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 2 0 0 0 2 

In [290]:
gp = GaussianProcessRegressor(kernel=kernels.DotProduct(), optimizer='fmin_l_bfgs_b' , alpha=1e-3, normalize_y=True)
gp.fit(X, Y)
X_test = Data['x']
X_test  = X_test.values.reshape((100, 1))
Y_test = Data['y']
Y_pred = gp.predict(X_test)
diff = Y_pred - Y_test
print(math.sqrt(np.dot(diff, diff)/100))
print(math.sqrt(np.dot(Y_test, Y_test)/100))


0.47182693004201387
0.7085801545780499


In [294]:
Correlations.shape[0]

3

In [292]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, T)
print(clf.predict(X))
print(T)

[0 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 2 1 0 0 0 2 0 1 2 0 0 0 2 0 0
 1 0 0 0 0 0 0 0 2 1 0 1 0]
[1 0 0 1 0 0 2 0 0 0 0 0 0 0 0 1 0 2 1 0 2 0 2 2 0 0 0 2 1 2 2 0 1 0 2 0 0
 1 1 0 0 0 0 0 1 1 1 0 2 0]
